In [ ]:
%load_ext autoreload
%autoreload 2

import scipy
from scipy.integrate import odeint
import numpy as np
import matplotlib.pyplot as plt
import random
import sir_model
import json
from random import randint

In [ ]:
# initialize recording of parameter choices and true/false


# USER: set bounds
beta_search_bounds = [0.1,0.25]
gamma_search_bounds = [0.01,0.04]

In [ ]:
# USER: list how many points for each parameter you'd like to synthesize

beta_values_to_synthesize = 50
gamma_values_to_synthesize = 50

In [ ]:
search_points_beta = np.linspace(beta_search_bounds[0], beta_search_bounds[1], beta_values_to_synthesize)
search_points_gamma = np.linspace(gamma_search_bounds[0], gamma_search_bounds[1], gamma_values_to_synthesize)

In [ ]:
def eval_point(beta_val, gamma_val, plot=False, rtol=1e-4):
    # parameters
    def beta(t): return np.piecewise(t, [t>=0], [beta_val])
    def gamma(t): return np.piecewise(t, [t>=0], [gamma_val])


    # USER: set initial conditions
    I0, R0 = 0.01, 0
    S0 = 1-I0-R0
    y0 = S0, I0, R0 # Initial conditions vector
    # USER: set simulation parameters
    dt = 5
    tstart = 0
    tend = 100
    tvect = np.arange(tstart, tend, dt)
    # simulate/solve ODEs
    sim = odeint(sir_model.SIR_model, y0, tvect, args=(beta, gamma), hmin=dt, mxstep=1)
    S, I, R = sim.T

    # print(list(zip(range(tstart, tend+1, dt), I)))

    # plot results - uncomment next line to plot time series.  not recommended for large numbers of points
    if plot:
        sir_model.plotSIR(tvect, S, I, R)
    # USER: write query condition.
    # query_condition =  0.15 <= max(I) <= 0.3  #((0.15 <= I[10]) and I[10] <= 1.0)
    # query_condition = (0.45 <= max(I)) and (max(I) <= 0.55) and (45 <= np.argmax(I)*dt) and (np.argmax(I)*dt <= 55)
    query_values = [0.45 <= I[int(t/dt)] <= 0.55 for t in range(tstart, tend+1, dt) if 45 <= t < 55]
    query_condition = all(query_values) 
    # print(I[9:11])
    query = '1' if query_condition else '0'
    param_assignments = {'beta': beta_val, 'gamma': gamma_val, 'assignment': query} # for "all", go through every option. for "any", only need one good parameter choice.
    return param_assignments, sim

# set parameters
def ps(param_synth_method, rtol=1e-3):
    param_choices_true_false = []
    for i in range(len(search_points_beta)):
        beta_val = search_points_beta[i]
        for j in range(len(search_points_gamma)):
            gamma_val = search_points_gamma[j]

            param_assignments, _ = eval_point(beta_val, gamma_val, rtol=rtol)
            param_choices_true_false.append(param_assignments)
            if param_synth_method == "any" and query == '1':
                return param_choices_true_false
    return param_choices_true_false
    


In [ ]:
param_choices_true_false = ps("all", rtol=1e-3)
print(param_choices_true_false)

id_number = randint(10**5, 10**6 - 1)

with open(f'sir_query_auto_2_param_{id_number}.json', 'w', encoding='utf-8') as f:
    json.dump(param_choices_true_false, f, ensure_ascii=False, indent=4)

In [ ]:
?odeint

In [ ]:
# my_params = [p for p in param_choices_true_false if p['assignment']=="1"][-1]
# param_assignment, sim = eval_point(my_params["beta"], my_params["gamma"], plot=True, rtol=1e-10)
param_assignment, sim = eval_point(0.25, 0.027755102040816326, plot=True, rtol=1e-10)
S, I, R = sim.T

print(list(enumerate(S))[10])
# print([S[i]-S[i-1] for i in range(1, len(S))])
print(my_params)


In [ ]:
# Plot "true/false" points.  Is there any way of characterizing these regions?

sir_model.plot_two_params("gamma", "beta", param_choices_true_false)



In [ ]:
print(param_choices_true_false)